# Network Flow Visualization Tutorial

This tutorial demonstrates how to use the visualization utilities in the `network-flow-solver` package to create publication-quality visualizations of network flow problems and solutions.

## Overview

The package provides three main visualization functions:

1. **`visualize_network()`** - Visualize the network structure with nodes, arcs, costs, and capacities
2. **`visualize_flows()`** - Visualize the flow solution with optional bottleneck highlighting
3. **`visualize_bottlenecks()`** - Focus on bottlenecks with utilization heatmaps

## Prerequisites

Make sure you have the visualization dependencies installed:

```bash
pip install 'network-flow-solver[visualization]'
```

This installs matplotlib and networkx, which are required for visualization.

## Setup and Imports

In [ ]:
# Import the necessary modules
# For inline display in Jupyter
import matplotlib.pyplot as plt

from network_solver import (
    build_problem,
    solve_min_cost_flow,
    visualize_bottlenecks,
    visualize_flows,
    visualize_network,
)

%matplotlib inline
plt.rcParams["figure.dpi"] = 100  # Adjust for your display

## Example 1: Simple Supply Chain Network

Let's start with a simple supply chain: two factories shipping to two warehouses.

In [ ]:
# Define a simple supply chain network
nodes = [
    {"id": "factory_a", "supply": 100},  # Factory A produces 100 units
    {"id": "factory_b", "supply": 80},  # Factory B produces 80 units
    {"id": "warehouse_1", "supply": -70},  # Warehouse 1 needs 70 units
    {"id": "warehouse_2", "supply": -110},  # Warehouse 2 needs 110 units
]

arcs = [
    {"tail": "factory_a", "head": "warehouse_1", "capacity": 80, "cost": 4.0},
    {"tail": "factory_a", "head": "warehouse_2", "capacity": 70, "cost": 6.0},
    {"tail": "factory_b", "head": "warehouse_1", "capacity": 50, "cost": 5.0},
    {"tail": "factory_b", "head": "warehouse_2", "capacity": 60, "cost": 3.0},
]

problem = build_problem(nodes, arcs, directed=True)

### Visualizing the Network Structure

Use `visualize_network()` to see the network structure with:
- **Green nodes**: Sources (supply > 0)
- **Red nodes**: Sinks (supply < 0)
- **Blue nodes**: Transshipment nodes (supply = 0)
- **Arc labels**: Show cost and capacity

In [ ]:
fig = visualize_network(problem, title="Simple Supply Chain Network")
plt.tight_layout()
plt.show()

### Solving and Visualizing the Flow Solution

In [ ]:
# Solve the problem
result = solve_min_cost_flow(problem)

print(f"Status: {result.status}")
print(f"Total Cost: {result.cost:.2f}")
print(f"Iterations: {result.iterations}")

### Visualizing Flows with Bottleneck Highlighting

Use `visualize_flows()` to see:
- Flow values on each arc (thickness proportional to flow)
- Bottlenecks highlighted in red (arcs at ≥90% capacity by default)
- Remaining capacity information

In [ ]:
fig = visualize_flows(
    problem,
    result,
    highlight_bottlenecks=True,
    title="Supply Chain Flow Solution with Bottlenecks",
)
plt.tight_layout()
plt.show()

## Example 2: Transportation Network with Multiple Layouts

Let's create a larger transportation network and explore different layout algorithms.

In [ ]:
# Create a 3x3 transportation problem
nodes = [
    # Sources
    {"id": "plant_1", "supply": 150},
    {"id": "plant_2", "supply": 200},
    {"id": "plant_3", "supply": 100},
    # Destinations
    {"id": "customer_1", "supply": -180},
    {"id": "customer_2", "supply": -120},
    {"id": "customer_3", "supply": -150},
]

# Create all possible shipping routes
arcs = []
costs = [
    [8, 6, 10],  # Plant 1 to customers
    [9, 12, 13],  # Plant 2 to customers
    [14, 9, 16],  # Plant 3 to customers
]

for i, plant in enumerate(["plant_1", "plant_2", "plant_3"]):
    for j, customer in enumerate(["customer_1", "customer_2", "customer_3"]):
        arcs.append(
            {
                "tail": plant,
                "head": customer,
                "capacity": 200,  # Large capacity
                "cost": costs[i][j],
            }
        )

transport_problem = build_problem(nodes, arcs, directed=True)
transport_result = solve_min_cost_flow(transport_problem)

print(f"Transportation Cost: ${transport_result.cost:.2f}")

### Comparing Different Layout Algorithms

The visualization functions support multiple layout algorithms:
- **spring**: Force-directed layout (default)
- **circular**: Nodes arranged in a circle
- **kamada_kawai**: Energy-minimization layout
- **planar**: Attempts planar layout (if graph is planar)

In [ ]:
# Create a figure with multiple subplots
layouts = ["spring", "circular", "kamada_kawai"]

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for ax, layout in zip(axes, layouts, strict=True):
    plt.sca(ax)
    visualize_flows(
        transport_problem,
        transport_result,
        layout=layout,
        figsize=(6, 5),
        title=f"{layout.replace('_', ' ').title()} Layout",
        show_zero_flows=False,
    )

plt.tight_layout()
plt.show()

## Example 3: Multi-Stage Supply Chain with Transshipment

Let's create a more complex network with intermediate distribution centers (transshipment nodes).

In [ ]:
# Multi-stage supply chain: Suppliers -> Distribution Centers -> Retailers
nodes = [
    # Suppliers
    {"id": "supplier_a", "supply": 200},
    {"id": "supplier_b", "supply": 150},
    # Distribution centers (transshipment)
    {"id": "dc_east", "supply": 0},
    {"id": "dc_west", "supply": 0},
    # Retailers
    {"id": "retail_1", "supply": -100},
    {"id": "retail_2", "supply": -120},
    {"id": "retail_3", "supply": -130},
]

arcs = [
    # Suppliers to DCs
    {"tail": "supplier_a", "head": "dc_east", "capacity": 120, "cost": 2.0},
    {"tail": "supplier_a", "head": "dc_west", "capacity": 100, "cost": 3.0},
    {"tail": "supplier_b", "head": "dc_east", "capacity": 80, "cost": 2.5},
    {"tail": "supplier_b", "head": "dc_west", "capacity": 90, "cost": 2.0},
    # DCs to Retailers
    {"tail": "dc_east", "head": "retail_1", "capacity": 80, "cost": 1.5},
    {"tail": "dc_east", "head": "retail_2", "capacity": 90, "cost": 1.8},
    {"tail": "dc_east", "head": "retail_3", "capacity": 70, "cost": 2.2},
    {"tail": "dc_west", "head": "retail_1", "capacity": 60, "cost": 2.0},
    {"tail": "dc_west", "head": "retail_2", "capacity": 70, "cost": 1.5},
    {"tail": "dc_west", "head": "retail_3", "capacity": 80, "cost": 1.6},
]

multi_stage_problem = build_problem(nodes, arcs, directed=True)

### Visualize the Network Structure

Notice the transshipment nodes (distribution centers) shown in blue.

In [ ]:
fig = visualize_network(
    multi_stage_problem,
    layout="spring",
    title="Multi-Stage Supply Chain Network",
    figsize=(14, 10),
)
plt.tight_layout()
plt.show()

In [ ]:
# Solve the multi-stage problem
multi_stage_result = solve_min_cost_flow(multi_stage_problem)

print(f"Status: {multi_stage_result.status}")
print(f"Total Cost: ${multi_stage_result.cost:.2f}")
print("\nFlow through distribution centers:")

# Calculate flow through each DC
for dc in ["dc_east", "dc_west"]:
    inflow = sum(flow for (tail, head), flow in multi_stage_result.flows.items() if head == dc)
    print(f"  {dc}: {inflow:.0f} units")

### Visualize the Flow Solution

In [ ]:
fig = visualize_flows(
    multi_stage_problem,
    multi_stage_result,
    highlight_bottlenecks=True,
    title="Multi-Stage Supply Chain Flow Solution",
    figsize=(14, 10),
)
plt.tight_layout()
plt.show()

## Example 4: Bottleneck Analysis

The `visualize_bottlenecks()` function provides a focused view of capacity constraints with a utilization heatmap.

In [ ]:
# Create a network with intentional bottlenecks
nodes = [
    {"id": "source_1", "supply": 100},
    {"id": "source_2", "supply": 80},
    {"id": "hub", "supply": 0},  # Bottleneck hub
    {"id": "dest_1", "supply": -90},
    {"id": "dest_2", "supply": -90},
]

arcs = [
    # Plenty of capacity to hub
    {"tail": "source_1", "head": "hub", "capacity": 150, "cost": 1.0},
    {"tail": "source_2", "head": "hub", "capacity": 150, "cost": 1.0},
    # Limited capacity from hub (bottleneck!)
    {"tail": "hub", "head": "dest_1", "capacity": 90, "cost": 2.0},
    {"tail": "hub", "head": "dest_2", "capacity": 90, "cost": 2.0},
]

bottleneck_problem = build_problem(nodes, arcs, directed=True)
bottleneck_result = solve_min_cost_flow(bottleneck_problem)

### Standard Flow Visualization

In [ ]:
fig = visualize_flows(
    bottleneck_problem,
    bottleneck_result,
    highlight_bottlenecks=True,
    bottleneck_threshold=0.9,  # 90% utilization
    title="Network with Bottlenecks",
)
plt.tight_layout()
plt.show()

### Focused Bottleneck Analysis

The `visualize_bottlenecks()` function shows:
- Only arcs above a utilization threshold (default 80%)
- Color gradient: Red (high utilization) → Yellow → Green (lower utilization)
- Flow and capacity information
- Utilization percentages

In [ ]:
fig = visualize_bottlenecks(
    bottleneck_problem,
    bottleneck_result,
    threshold=0.8,  # Show arcs with ≥80% utilization
    title="Bottleneck Analysis (≥80% Utilization)",
)
plt.tight_layout()
plt.show()

### Adjusting the Bottleneck Threshold

You can adjust the threshold to see different levels of utilization.

In [ ]:
# Show even lower utilization arcs
fig = visualize_bottlenecks(
    bottleneck_problem,
    bottleneck_result,
    threshold=0.5,  # Show arcs with ≥50% utilization
    title="Bottleneck Analysis (≥50% Utilization)",
)
plt.tight_layout()
plt.show()

## Example 5: Customization Options

All visualization functions support extensive customization.

In [ ]:
# Create a simple network for demonstration
nodes = [
    {"id": "A", "supply": 50},
    {"id": "B", "supply": -20},
    {"id": "C", "supply": -30},
]

arcs = [
    {"tail": "A", "head": "B", "capacity": 25, "cost": 2.0},
    {"tail": "A", "head": "C", "capacity": 30, "cost": 3.0},
]

simple_problem = build_problem(nodes, arcs, directed=True)
simple_result = solve_min_cost_flow(simple_problem)

### Customization Options

In [ ]:
# Customize the visualization
fig = visualize_flows(
    simple_problem,
    simple_result,
    layout="circular",  # Layout algorithm
    figsize=(10, 8),  # Figure size
    node_size=2000,  # Larger nodes
    font_size=12,  # Larger font
    highlight_bottlenecks=True,
    bottleneck_threshold=0.85,  # Custom threshold
    show_zero_flows=True,  # Show arcs with zero flow
    title="Customized Visualization",
)
plt.tight_layout()
plt.show()

### Hiding Arc Labels for Cleaner Visuals

In [ ]:
# Network visualization without arc labels
fig = visualize_network(
    multi_stage_problem,
    show_arc_labels=False,  # Hide cost/capacity labels
    title="Clean Network Structure (No Arc Labels)",
    figsize=(12, 8),
)
plt.tight_layout()
plt.show()

## Example 6: Saving Visualizations

You can save the figures to files for use in reports or presentations.

In [ ]:
# Create a visualization and save it
fig = visualize_flows(
    multi_stage_problem,
    multi_stage_result,
    title="Multi-Stage Supply Chain",
    figsize=(14, 10),
)

# Save in multiple formats
fig.savefig("supply_chain_flows.png", dpi=300, bbox_inches="tight")
fig.savefig("supply_chain_flows.pdf", bbox_inches="tight")  # Vector format

print("Figures saved!")
plt.show()

## Summary

This tutorial covered:

1. **Network Structure Visualization** - `visualize_network()` shows the network topology with color-coded nodes
2. **Flow Solution Visualization** - `visualize_flows()` displays the optimal flow with bottleneck highlighting
3. **Bottleneck Analysis** - `visualize_bottlenecks()` focuses on capacity-constrained arcs with utilization heatmaps
4. **Layout Options** - Multiple algorithms (spring, circular, kamada_kawai, planar)
5. **Customization** - Extensive options for figure size, node size, fonts, colors, and thresholds
6. **Saving Figures** - Export to PNG, PDF, or other formats

## Next Steps

- Try visualizing your own network flow problems
- Experiment with different layout algorithms to find the best representation
- Use bottleneck analysis to identify capacity constraints in your networks
- Combine visualizations with the solver's warm start feature to explore "what-if" scenarios

For more information, see:
- [API Documentation](../docs/api.md)
- [Examples](../docs/examples.md)
- [Main Tutorial](network_flow_tutorial.ipynb)